In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from lls import LLS
from train_test_split import TrainTestSplit

In [2]:
data = pd.read_csv('Inputs\HousePrice.csv')
data.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33


Preprocess data and update dollar price to july 2023.



In [3]:
data['Parking'].replace([True, False], [0, 1], inplace=True)
data['Warehouse'].replace([True, False], [0, 1], inplace=True)
data['Elevator'].replace([True, False], [0, 1], inplace=True)

In [4]:
data['Price(USD)'] = data['Price'] / 49000
data.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,0,0,0,Shahran,1.850000e+09,37755.102041
1,60,1,0,0,0,Shahran,1.850000e+09,37755.102041
2,79,2,0,0,0,Pardis,5.500000e+08,11224.489796
3,95,2,0,0,0,Shahrake Qods,9.025000e+08,18418.367347
4,123,2,0,0,0,Shahrake Gharb,7.000000e+09,142857.142857


Show the address of the 5 most expensive houses


In [5]:
price_sorted = data.sort_values(by=['Price'], ascending=False).reset_index(drop=True)
price_sorted.head(5)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,420,4,0,0,0,Zaferanieh,9.240000e+10,1.885714e+06
1,705,5,0,0,1,Abazar,9.100000e+10,1.857143e+06
2,400,5,0,0,1,Lavasan,8.500000e+10,1.734694e+06
3,680,5,0,0,1,Ekhtiarieh,8.160000e+10,1.665306e+06
4,350,4,0,0,0,Niavaran,8.050000e+10,1.642857e+06


Use all possible features for X_train

In [6]:
X = np.array(data[['Area', 'Room', 'Parking', 'Warehouse', 'Elevator']])
Y = np.array(data[['Price']])

In [7]:
data.drop('Address', axis=1, inplace=True)
data.head()

,Area,Room,Parking,Warehouse,Elevator,Price,Price(USD)
0,63,1,0,0,0,1.850000e+09,37755.102041
1,60,1,0,0,0,1.850000e+09,37755.102041
2,79,2,0,0,0,5.500000e+08,11224.489796
3,95,2,0,0,0,9.025000e+08,18418.367347
4,123,2,0,0,0,7.000000e+09,142857.142857


Split your dataset to train and test with train_test_split function

In [8]:
X_train, X_test, Y_train, Y_test = TrainTestSplit(X, Y, test_size=0.2, shuffle=True)



Fit the LLS model on your training dataset


In [9]:
lls = LLS()
lls.fit(X_train, Y_train)

array([[ 8.30411179e+07],
       [-1.10185793e+09],
       [-1.04203948e+09],
       [-1.54065292e+09],
       [-1.84244599e+09]])

Evaluate your model on your test dataset using MAE, MSE and RMSE loss functions. 

In [10]:
lls.evaluate(X_test, Y_test, 'mae')


28884444360.422688

In [11]:
lls.evaluate(X_test, Y_test, 'mse')

4.4636569080944755e+20

In [12]:
lls.evaluate(X_test, Y_test, 'rmse')

21127368288.77292

## Why the MSE metric is a very large number?
* Because the numbers of the differences have been raised to the power of 2 to get out of the negative state, but at the end, no root has been taken from them.

Compare your result with Scikit-Learn's results

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV 

In [14]:
lls_sk= LinearRegression()
lls_sk.fit(X_train,Y_train)

LinearRegression()

In [15]:
Y_pred = lls_sk.predict(X_test)
lls.evaluate_outer_model(Y_pred, Y_test, metric='mae')

2714691034.9825993

In [16]:
lls.evaluate_outer_model(Y_pred, Y_test, metric='mse')

2.964363840605792e+19

In [17]:
lls.evaluate_outer_model(Y_pred, Y_test, metric='rmse')

5444597175.738341

In [18]:
lls_sk= RidgeCV()
lls_sk.fit(X_train,Y_train)

RidgeCV()

In [19]:
Y_pred = lls_sk.predict(X_test)
lls.evaluate_outer_model(Y_pred, Y_test, metric='mae')

2711274484.027565

In [20]:
lls.evaluate_outer_model(Y_pred, Y_test, metric='mse')

2.9659570977522565e+19

In [21]:
lls.evaluate_outer_model(Y_pred, Y_test, metric='rmse')

5446060133.48389